# 
# Utilização de Algorítmos de Machine Learning para Identificação de Empresas "de Fachada" em Operações de Importação
TCC PUC Minas - LUCIANA MACEDO RODRIGUES

### NOTEBOOK 1 - COLETA DOS DADOS

### 1 - Configurações iniciais

In [1]:
# Importando os pacotes necessários
import pandas as pd
import numpy as np

In [2]:
# Versão do pandas em uso
pd.__version__

'1.4.2'

### 2 - Leitura das bases de dados e verificações preliminares

In [3]:
dadosADU = pd.read_csv('arquivos/dadosADU.csv', sep = ';')

In [4]:
dadosCNPJ = pd.read_csv('arquivos/dadosCNPJ.csv', sep = ';')

In [5]:
dadosDIRF = pd.read_csv('arquivos/dadosDIRF.csv', sep = ';')

In [6]:
dadosRB = pd.read_csv('arquivos/dadosRB.csv', sep = ';')

In [7]:
# Linhas iniciais
dadosADU.head()

,ID_EMP,UF_EMP,ANO SIT CAD EMP ADQUIR,SIT CAD EMP ADQUIR,MOTIVO SIT CAD EMP ADQUIR,IMP_VOL,IMP_VAL,IMP_PESO,IMP_CUSTO
0,807,SP,2004.0,Ativa,NÃO INFORMADO,64,9211744,169,11017585
1,1392,SP,2005.0,Ativa,NÃO INFORMADO,1,31117,1,46250
2,1444,SP,2021.0,Baixada,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,8,554300,179,716987
3,1596,SP,2005.0,Ativa,NÃO INFORMADO,39,4560342,221,6138632
4,1904,SP,2002.0,Ativa,NÃO INFORMADO,1,118759,24,158602


In [8]:
# Tipos de Dados
dadosADU.dtypes

ID_EMP                         int64
UF_EMP                        object
ANO SIT CAD EMP ADQUIR       float64
SIT CAD EMP ADQUIR            object
MOTIVO SIT CAD EMP ADQUIR     object
IMP_VOL                        int64
IMP_VAL                        int64
IMP_PESO                       int64
IMP_CUSTO                      int64
dtype: object

In [9]:
# Quantidade de (linhas,colunas)
dadosADU.shape

(20953, 9)

In [10]:
# Dados faltantes
dadosADU.isnull().sum()

ID_EMP                         0
UF_EMP                         0
ANO SIT CAD EMP ADQUIR       548
SIT CAD EMP ADQUIR             0
MOTIVO SIT CAD EMP ADQUIR      0
IMP_VOL                        0
IMP_VAL                        0
IMP_PESO                       0
IMP_CUSTO                      0
dtype: int64

In [11]:
dadosCNPJ.head()

,ID_EMP2,DT_ABERT_EMP,EMP_PORTE,Empresa - 6 Dia Sit. Cad. Atual (ID),Empresa - Sit. Cad. Atual (Nome),Empresa - Sit. Cad. Motivo Atual (Nome),EMP_CS_MAX,EMP_CS_MIN
0,807,16/05/1994,DEMAIS,28/08/2004,ATIVA,SEM MOTIVO,50000,50000
1,1392,06/05/1994,DEMAIS,03/11/2005,ATIVA,SEM MOTIVO,23000000,15000000
2,1444,09/05/1994,DEMAIS,27/05/2021,BAIXADA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,0,0
3,1596,09/05/1994,DEMAIS,25/03/2005,ATIVA,SEM MOTIVO,300000,300000
4,1904,09/05/1994,DEMAIS,09/11/2002,ATIVA,SEM MOTIVO,600000,600000


In [12]:
dadosCNPJ.dtypes

ID_EMP2                                     int64
DT_ABERT_EMP                               object
EMP_PORTE                                  object
Empresa - 6 Dia Sit. Cad. Atual (ID)       object
Empresa - Sit. Cad. Atual (Nome)           object
Empresa - Sit. Cad. Motivo Atual (Nome)    object
EMP_CS_MAX                                  int64
EMP_CS_MIN                                  int64
dtype: object

In [13]:
dadosCNPJ.shape

(20954, 8)

In [14]:
dadosCNPJ.isnull().sum()

ID_EMP2                                      0
DT_ABERT_EMP                                 1
EMP_PORTE                                    0
Empresa - 6 Dia Sit. Cad. Atual (ID)       555
Empresa - Sit. Cad. Atual (Nome)             0
Empresa - Sit. Cad. Motivo Atual (Nome)      0
EMP_CS_MAX                                   0
EMP_CS_MIN                                   0
dtype: int64

In [15]:
dadosDIRF.head()

,ID_EMP3,EMP_EMPREG
0,807,32
1,1392,256
2,1444,26
3,1596,14
4,1904,102


In [16]:
dadosDIRF.dtypes

ID_EMP3       int64
EMP_EMPREG    int64
dtype: object

In [17]:
dadosDIRF.shape

(19016, 2)

In [18]:
dadosDIRF.isnull().sum()

ID_EMP3       0
EMP_EMPREG    0
dtype: int64

In [19]:
dadosRB.head()

,ID_EMP4,EMP_REC
0,807,26309539
1,1392,28811802
2,1444,2526911
3,1596,37290202
4,1904,60054654


In [20]:
dadosRB.dtypes

ID_EMP4     int64
EMP_REC    object
dtype: object

In [21]:
dadosRB.shape

(20646, 2)

In [22]:
dadosRB.isnull().sum()

ID_EMP4    0
EMP_REC    0
dtype: int64

### 3 -  Junção das bases de dados

As bases de dados da DIRF e da RB não retornaram dados para todos os CNPJ consultados (evidenciado pela quantidade de linhas inferior à de dadosADU dessas bases), portanto existirão dados em branco na tabela resultante do join. 

In [23]:
# Renomeando as colunas CNPJ de todos as bases de dados para padronizar
dadosADU.rename(columns={'ID_EMP':'ID'}, 
             inplace=True)
dadosCNPJ.rename(columns={'ID_EMP2':'ID'}, 
             inplace=True)
dadosDIRF.rename(columns={'ID_EMP3':'ID'}, 
             inplace=True)
dadosRB.rename(columns={'ID_EMP4':'ID'}, 
             inplace=True)

In [24]:
# Verificando alterações em cada base de dados
dadosADU.dtypes

ID                             int64
UF_EMP                        object
ANO SIT CAD EMP ADQUIR       float64
SIT CAD EMP ADQUIR            object
MOTIVO SIT CAD EMP ADQUIR     object
IMP_VOL                        int64
IMP_VAL                        int64
IMP_PESO                       int64
IMP_CUSTO                      int64
dtype: object

In [25]:
dadosCNPJ.dtypes

ID                                          int64
DT_ABERT_EMP                               object
EMP_PORTE                                  object
Empresa - 6 Dia Sit. Cad. Atual (ID)       object
Empresa - Sit. Cad. Atual (Nome)           object
Empresa - Sit. Cad. Motivo Atual (Nome)    object
EMP_CS_MAX                                  int64
EMP_CS_MIN                                  int64
dtype: object

In [26]:
dadosDIRF.dtypes

ID            int64
EMP_EMPREG    int64
dtype: object

In [27]:
dadosRB.dtypes

ID          int64
EMP_REC    object
dtype: object

In [28]:
# O dataframe dadosCNPJ apresenta 1 linha a mais que o dadosADU. 
# Usando a função value_counts() confirmamos a suspeita de dados em duplicidade para um mesmo CNPJ
dadosCNPJ.ID.value_counts()

23809368    2
807         1
27928707    1
27953984    1
27945967    1
           ..
9334281     1
9334062     1
9330699     1
9328663     1
97837181    1
Name: ID, Length: 20953, dtype: int64

In [29]:
# Localizando o índice das linhas de dadosCNPJ que contém o mesmo valor para avaliar a melhor opção para eliminar
np.where(dadosCNPJ["ID"] == 23809368)

(array([12896, 12897], dtype=int64),)

In [30]:
# São dados duplicados - dois registros em 2015 e 2017 da mesma empresa, eliminaremos o segundo na posição 12897
dadosCNPJ.drop(index=[12897],inplace=True)

# Verificando se a repetição foi eliminada
np.where(dadosCNPJ["ID"] == 23809368)

(array([12896], dtype=int64),)

In [31]:
# Juntando usando CNPJ como chave e left a partir de ADU>CNPJ>DIRF>RB (do maior volume de cnpj para o menor)
df_merged = pd.merge (dadosADU, dadosCNPJ, how = 'left', on = 'ID')
df_merged = pd.merge (df_merged, dadosDIRF, how = 'left', on = 'ID')
df_merged = pd.merge (df_merged, dadosRB, how = 'left', on = 'ID')

In [32]:
# Obtendo todas as informações de df_merged
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20953 entries, 0 to 20952
Data columns (total 18 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   ID                                       20953 non-null  int64  
 1   UF_EMP                                   20953 non-null  object 
 2   ANO SIT CAD EMP ADQUIR                   20405 non-null  float64
 3   SIT CAD EMP ADQUIR                       20953 non-null  object 
 4   MOTIVO SIT CAD EMP ADQUIR                20953 non-null  object 
 5   IMP_VOL                                  20953 non-null  int64  
 6   IMP_VAL                                  20953 non-null  int64  
 7   IMP_PESO                                 20953 non-null  int64  
 8   IMP_CUSTO                                20953 non-null  int64  
 9   DT_ABERT_EMP                             20952 non-null  object 
 10  EMP_PORTE                                20953

In [33]:
df_merged.head()

,ID,UF_EMP,ANO SIT CAD EMP ADQUIR,SIT CAD EMP ADQUIR,MOTIVO SIT CAD EMP ADQUIR,IMP_VOL,IMP_VAL,IMP_PESO,IMP_CUSTO,DT_ABERT_EMP,EMP_PORTE,Empresa - 6 Dia Sit. Cad. Atual (ID),Empresa - Sit. Cad. Atual (Nome),Empresa - Sit. Cad. Motivo Atual (Nome),EMP_CS_MAX,EMP_CS_MIN,EMP_EMPREG,EMP_REC
0,807,SP,2004.0,Ativa,NÃO INFORMADO,64,9211744,169,11017585,16/05/1994,DEMAIS,28/08/2004,ATIVA,SEM MOTIVO,50000,50000,32.0,26309539
1,1392,SP,2005.0,Ativa,NÃO INFORMADO,1,31117,1,46250,06/05/1994,DEMAIS,03/11/2005,ATIVA,SEM MOTIVO,23000000,15000000,256.0,28811802
2,1444,SP,2021.0,Baixada,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,8,554300,179,716987,09/05/1994,DEMAIS,27/05/2021,BAIXADA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,0,0,26.0,2526911
3,1596,SP,2005.0,Ativa,NÃO INFORMADO,39,4560342,221,6138632,09/05/1994,DEMAIS,25/03/2005,ATIVA,SEM MOTIVO,300000,300000,14.0,37290202
4,1904,SP,2002.0,Ativa,NÃO INFORMADO,1,118759,24,158602,09/05/1994,DEMAIS,09/11/2002,ATIVA,SEM MOTIVO,600000,600000,102.0,60054654


### 4 - Formatação da base de dados df_merged
A base de dados resultante da junção das fontes de dados originais apresenta algumas colunas com dados desnecessários para o estudo, que eliminaremos a seguir.

As colunas 'SIT CAD EMP ADQUIR', 'MOTIVO SIT CAD EMP ADQUIR''Empresa - Sit. Cad. Atual (Nome)' e 'Empresa - Sit. Cad. Motivo Atual (Nome)' possuem dados semelhantes, porém extraídos em momentos diferentes, podendo apresentar divergências. Para verificar quais dados seriam mais interessantes para o estudo, optamos por analisá-los após a junção das bases.

Por fim, antes de passar à etapa de tratamento dos dados propriamente dita, reorganizaremos a ordem das colunas e as renomearemos.

In [34]:
# Eliminando as colunas desnecessárias
df_merged.drop(columns=["ANO SIT CAD EMP ADQUIR", "ID", "EMP_CS_MIN"],inplace=True)
df_merged.head()

,UF_EMP,SIT CAD EMP ADQUIR,MOTIVO SIT CAD EMP ADQUIR,IMP_VOL,IMP_VAL,IMP_PESO,IMP_CUSTO,DT_ABERT_EMP,EMP_PORTE,Empresa - 6 Dia Sit. Cad. Atual (ID),Empresa - Sit. Cad. Atual (Nome),Empresa - Sit. Cad. Motivo Atual (Nome),EMP_CS_MAX,EMP_EMPREG,EMP_REC
0,SP,Ativa,NÃO INFORMADO,64,9211744,169,11017585,16/05/1994,DEMAIS,28/08/2004,ATIVA,SEM MOTIVO,50000,32.0,26309539
1,SP,Ativa,NÃO INFORMADO,1,31117,1,46250,06/05/1994,DEMAIS,03/11/2005,ATIVA,SEM MOTIVO,23000000,256.0,28811802
2,SP,Baixada,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,8,554300,179,716987,09/05/1994,DEMAIS,27/05/2021,BAIXADA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,0,26.0,2526911
3,SP,Ativa,NÃO INFORMADO,39,4560342,221,6138632,09/05/1994,DEMAIS,25/03/2005,ATIVA,SEM MOTIVO,300000,14.0,37290202
4,SP,Ativa,NÃO INFORMADO,1,118759,24,158602,09/05/1994,DEMAIS,09/11/2002,ATIVA,SEM MOTIVO,600000,102.0,60054654


In [35]:
# Verificando se há divergência entre os dados de 'SIT CAD EMP ADQUIR' e 'Empresa - Sit. Cad. Atual (Nome)'
freq = df_merged.groupby(['SIT CAD EMP ADQUIR', 'MOTIVO SIT CAD EMP ADQUIR']).size()
freq2 = df_merged.groupby(['Empresa - Sit. Cad. Atual (Nome)', 'Empresa - Sit. Cad. Motivo Atual (Nome)']).size()
freq

SIT CAD EMP ADQUIR  MOTIVO SIT CAD EMP ADQUIR                         
Ativa               NÃO INFORMADO                                         20062
Baixada             ENCERRAMENTO DA LIQUIDACAO                                1
                    EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA         442
                    INCORPORACAO                                            205
                    INEXISTENTE DE FATO                                      14
Inapta              INEXISTENTE DE FATO                                       1
                    LOCALIZACAO DESCONHECIDA                                  6
                    OMISSAO DE DECLARACOES                                  162
                    PRATICA IRREGULAR DE OPERACAO DE COMERCIO EXTERIOR       37
Suspensa            INCONSISTÊNCIA CADASTRAL                                  3
                    INEXISTENTE DE FATO                                       1
                    INTERRUPCAO TEMPORARIA DAS AT

In [36]:
freq2

Empresa - Sit. Cad. Atual (Nome)  Empresa - Sit. Cad. Motivo Atual (Nome)           
ATIVA                             SEM MOTIVO                                            20178
BAIXADA                           EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA         418
                                  INCORPORACAO                                            196
                                  INEXISTENTE DE FATO                                      14
INAPTA                            LOCALIZACAO DESCONHECIDA                                  7
                                  OMISSAO DE DECLARACOES                                   80
                                  PRATICA IRREGULAR DE OPERACAO DE COMERCIO EXTERIOR       35
SUSPENSA                          INCONSISTENCIA CADASTRAL                                  4
                                  INEXISTENTE DE FATO                                       2
                                  INTERRUPCAO TEMPORARIA DAS ATIVIDAD

In [37]:
# Há divergências, porém para fins do estudo, os motivos de interesse sofrem pouca alteração entre as duas bases.
# Como 'Empresa - Sit. Cad. Atual (Nome)' representa a situação cadastral ATUAL da empresa obtida 
# a partir do DATASET Cadastro CNPJ optamos por mantê-la, eliminando as colunas "SIT CAD EMP ADQUIR" e
# "MOTIVO SIT CAD EMP ADQUIR"
df_merged.drop(columns=["SIT CAD EMP ADQUIR", "MOTIVO SIT CAD EMP ADQUIR"],inplace=True)
df_merged.head()

,UF_EMP,IMP_VOL,IMP_VAL,IMP_PESO,IMP_CUSTO,DT_ABERT_EMP,EMP_PORTE,Empresa - 6 Dia Sit. Cad. Atual (ID),Empresa - Sit. Cad. Atual (Nome),Empresa - Sit. Cad. Motivo Atual (Nome),EMP_CS_MAX,EMP_EMPREG,EMP_REC
0,SP,64,9211744,169,11017585,16/05/1994,DEMAIS,28/08/2004,ATIVA,SEM MOTIVO,50000,32.0,26309539
1,SP,1,31117,1,46250,06/05/1994,DEMAIS,03/11/2005,ATIVA,SEM MOTIVO,23000000,256.0,28811802
2,SP,8,554300,179,716987,09/05/1994,DEMAIS,27/05/2021,BAIXADA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,0,26.0,2526911
3,SP,39,4560342,221,6138632,09/05/1994,DEMAIS,25/03/2005,ATIVA,SEM MOTIVO,300000,14.0,37290202
4,SP,1,118759,24,158602,09/05/1994,DEMAIS,09/11/2002,ATIVA,SEM MOTIVO,600000,102.0,60054654


In [38]:
# Organizando as colunas no df_merged
df_merged = df_merged[['UF_EMP', 'EMP_PORTE', 'EMP_CS_MAX', 'IMP_VOL',
                       'IMP_VAL', 'IMP_PESO', 'IMP_CUSTO', 'EMP_EMPREG', 'EMP_REC', 'DT_ABERT_EMP', 
                       'Empresa - 6 Dia Sit. Cad. Atual (ID)', 'Empresa - Sit. Cad. Atual (Nome)', 
                       'Empresa - Sit. Cad. Motivo Atual (Nome)']]
df_merged.head()

,UF_EMP,EMP_PORTE,EMP_CS_MAX,IMP_VOL,IMP_VAL,IMP_PESO,IMP_CUSTO,EMP_EMPREG,EMP_REC,DT_ABERT_EMP,Empresa - 6 Dia Sit. Cad. Atual (ID),Empresa - Sit. Cad. Atual (Nome),Empresa - Sit. Cad. Motivo Atual (Nome)
0,SP,DEMAIS,50000,64,9211744,169,11017585,32.0,26309539,16/05/1994,28/08/2004,ATIVA,SEM MOTIVO
1,SP,DEMAIS,23000000,1,31117,1,46250,256.0,28811802,06/05/1994,03/11/2005,ATIVA,SEM MOTIVO
2,SP,DEMAIS,0,8,554300,179,716987,26.0,2526911,09/05/1994,27/05/2021,BAIXADA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA
3,SP,DEMAIS,300000,39,4560342,221,6138632,14.0,37290202,09/05/1994,25/03/2005,ATIVA,SEM MOTIVO
4,SP,DEMAIS,600000,1,118759,24,158602,102.0,60054654,09/05/1994,09/11/2002,ATIVA,SEM MOTIVO


In [39]:
# Listando as colunas para renomear
list(df_merged.columns)

['UF_EMP',
 'EMP_PORTE',
 'EMP_CS_MAX',
 'IMP_VOL',
 'IMP_VAL',
 'IMP_PESO',
 'IMP_CUSTO',
 'EMP_EMPREG',
 'EMP_REC',
 'DT_ABERT_EMP',
 'Empresa - 6 Dia Sit. Cad. Atual (ID)',
 'Empresa - Sit. Cad. Atual (Nome)',
 'Empresa - Sit. Cad. Motivo Atual (Nome)']

In [40]:
# Renomeando as colunas
df_merged.rename(columns={'Empresa - 6 Dia Sit. Cad. Atual (ID)' : 'DT_SIT_CAD_EMP', 
                          'Empresa - Sit. Cad. Atual (Nome)': 'SIT_CAD_EMP', 
                          'Empresa - Sit. Cad. Motivo Atual (Nome)': 'MOT_SIT_CAD_EMP'}, 
             inplace=True)
list(df_merged.columns)

['UF_EMP',
 'EMP_PORTE',
 'EMP_CS_MAX',
 'IMP_VOL',
 'IMP_VAL',
 'IMP_PESO',
 'IMP_CUSTO',
 'EMP_EMPREG',
 'EMP_REC',
 'DT_ABERT_EMP',
 'DT_SIT_CAD_EMP',
 'SIT_CAD_EMP',
 'MOT_SIT_CAD_EMP']

In [41]:
# Criando csv de df_merged
df_merged.to_csv("arquivos/df_merged.csv", encoding = 'utf-8', index = False)

In [42]:
# Verificando csv de df_merged
df_merged = pd.read_csv('arquivos/df_merged.csv', sep = ',')
df_merged.head()

,UF_EMP,EMP_PORTE,EMP_CS_MAX,IMP_VOL,IMP_VAL,IMP_PESO,IMP_CUSTO,EMP_EMPREG,EMP_REC,DT_ABERT_EMP,DT_SIT_CAD_EMP,SIT_CAD_EMP,MOT_SIT_CAD_EMP
0,SP,DEMAIS,50000,64,9211744,169,11017585,32.0,26309539,16/05/1994,28/08/2004,ATIVA,SEM MOTIVO
1,SP,DEMAIS,23000000,1,31117,1,46250,256.0,28811802,06/05/1994,03/11/2005,ATIVA,SEM MOTIVO
2,SP,DEMAIS,0,8,554300,179,716987,26.0,2526911,09/05/1994,27/05/2021,BAIXADA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA
3,SP,DEMAIS,300000,39,4560342,221,6138632,14.0,37290202,09/05/1994,25/03/2005,ATIVA,SEM MOTIVO
4,SP,DEMAIS,600000,1,118759,24,158602,102.0,60054654,09/05/1994,09/11/2002,ATIVA,SEM MOTIVO
